In [1]:
%%capture
import pyterrier as pt
if not pt.started():
  pt.init()

In [2]:
qrels_path=("/content/drive/MyDrive/workspace/qrels.qrels")
qrels = pt.io.read_qrels(qrels_path)

In [8]:
import pandas as pd
df = pd.read_csv('/content/drive/MyDrive/workspace/tagged_corpus.csv')

from xml.etree import ElementTree
tree = ElementTree.parse('/content/drive/MyDrive/workspace/topics.xml')
data = tree.getroot()
titles = []
title_ids = []
for topic in data:
    titles.append(topic.find('title').text.lower()[:-1])
    num = topic.find('number').text
    title_ids.append(num)
topics = pd.DataFrame(titles,columns = ['query'])
topics['qid'] = topics.index + 51
columns_titles = ["qid","query"]
topics=topics.reindex(columns=columns_titles)

qrels_path=("/content/drive/MyDrive/workspace/qrels.qrels")
qrels = pt.io.read_qrels(qrels_path)

docnos = qrels.docno


In [4]:
from nltk.tokenize import RegexpTokenizer
tokenizer = RegexpTokenizer(r'\w+')

In [22]:
df = df.fillna('')
df['tagged'] = df['claims'] + " " + df['premises']

In [27]:
df.head()

,Unnamed: 0,docno,text,claims,premises,tagged
0,0,S1f6b58eb-A5c530110,"This week I have my hair in braids, much like ...",,I have my hair in braids much like I have had ...,I have my hair in braids much like I have had...
1,1,S1a9db4fc-Acc4206f5,"On November 10, UN Women elected its first exe...",,On November 10 UN Women elected its first exec...,On November 10 UN Women elected its first exe...
2,2,S1a924cd4-A994e77ca,"Madam, October is Women's History Month and al...",Research want disease,Madam October is Women 's History Month and al...,Research want disease Madam October is Women '...
3,3,S1a3be6ac-A873cd7aa,"I rise today to recognize Frederick Couchie, f...",,rise today to recognize Frederick Couchie fath...,rise today to recognize Frederick Couchie fat...
4,4,S1beff81f-A1206f3ee,I find the continued existence of and the incr...,,I find the continued existence of and the incr...,I find the continued existence of and the inc...


In [23]:
pd_indexer = pt.DFIndexer("./pd_index")
indexref = pd_indexer.index(df["tagged"], df["docno"])

index = pt.IndexFactory.of(indexref)
retr = pt.BatchRetrieve(index, controls = {"wmodel": "TF_IDF"})
res=retr.transform(topics)

res

,qid,docid,docno,rank,score,query
0,51,279241,Sd32774d-A5bbc95,0,15.656952,do we need sex education in schools
1,51,330916,S6353b9ac-A2df6348e,1,15.444213,do we need sex education in schools
2,51,4203,Sf27da082-Abdd3fe78,2,15.086112,do we need sex education in schools
3,51,213692,S11b834b9-Aabe96a35,3,14.955171,do we need sex education in schools
4,51,4198,Sf27da082-A764ccfdc,4,14.906049,do we need sex education in schools
...,...,...,...,...,...,...
49995,100,333938,S2b54df17-A6025fb9f,995,3.030660,do we need cash
49996,100,229453,S95dc4242-A41c885cd,996,3.030446,do we need cash
49997,100,162555,S7ef42de7-A7f871ab3,997,3.029360,do we need cash
49998,100,31921,Scf73b6a8-Ad5d541ce,998,3.026631,do we need cash


In [24]:
eval = pt.Utils.evaluate(res,qrels,metrics=["ndcg_cut_5", "bpref", "ndcg_cut_25"])
eval

{'bpref': 0.6137173057820225,
 'ndcg_cut_25': 0.41654041771600414,
 'ndcg_cut_5': 0.4918236677637124}